# Work on image

In [8]:
def draw_landmarks(image, landmarks):
    radius = 5

    # Check if image width is greater than 1000 px.
    # To improve visualization.
    if image.shape[1] > 1000:
        radius = 8

    for idx, kpt_data in enumerate(landmarks):
        loc_x, loc_y = np.array(kpt_data[:2], dtype=int).tolist()
        color_id = list(COLORS_RGB_MAP[int(kpt_data[-1])].values())[0]

        cv2.circle(image,
                   (loc_x, loc_y),
                   radius,
                   color=color_id[::-1],
                   thickness=-1,
                   lineType=cv2.LINE_AA)

      # Draw keypoint number
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 0.5
    font_thickness = 1
    # text = str(int(kpt_data[-1]))
    text = f"{int(kpt_data[-1])}: ({loc_x}, {loc_y})"
    text_size = cv2.getTextSize(text, font, font_scale, font_thickness)[0]
    text_x = loc_x - text_size[0] // 2
    text_y = loc_y - radius - 5
    print('************* text:', text)
    cv2.putText(image, text, (text_x, text_y), font, font_scale, color=(255, 255, 255), thickness=font_thickness)

    return image

In [7]:
def resize_with_aspect_ratio(img, new_width=None, new_height=None):
    # Get the current height and width
    height, width = img.shape[:2]

    # If only width is specified
    if new_width is not None and new_height is None:
        # Calculate the aspect ratio and new height
        aspect_ratio = width / height
        new_height = int(new_width / aspect_ratio)

    # If only height is specified
    elif new_height is not None and new_width is None:
        # Calculate the aspect ratio and new width
        aspect_ratio = height / width
        new_width = int(new_height / aspect_ratio)

    # If both width and height are specified, ignore aspect ratio
    elif new_width is not None and new_height is not None:
        pass

    # Resize the image
    resized_img = cv2.resize(img, (new_width, new_height))
    return resized_img

In [4]:
############################### cap working ############################### 
def wear_cap(cap,animal_image,  filter_kpts):
    

    ################# Check two points (if 14 and 15 exist in the third item of each list)
    
    point_14_exists = any(14 == sublist[2] for sublist in filter_kpts)
    point_15_exists = any(15 == sublist[2] for sublist in filter_kpts)


    

    ################ check four points 

    # Points to check
    points_to_check = [14, 15, 19, 18]
    
    # Check if all points exist in the third item of any list
    all_points_exist = all(point in [sublist[2] for sublist in filter_kpts] for point in points_to_check)
    print(f'----------****** four points exits ******----------{all_points_exist}', )
    if all_points_exist:
        print("************ All points exist.")
        

        ## *********** check if 19 > 15 and 18 > 14 (condition to correct image flip )
        # get value of 19 and 15
        point_19 = next(sublist for sublist in filter_kpts if sublist[2] == 19)
        point_15 = next(sublist for sublist in filter_kpts if sublist[2] == 15)

        if point_19[1] > point_15[1]:
            
            
    
    
            # get value of 18 and 14
            point_18 = next(sublist for sublist in filter_kpts if sublist[2] == 18)
            point_14 = next(sublist for sublist in filter_kpts if sublist[2] == 14)

            # Calculate the width between points 14 and 15
            width_19_15 = np.sqrt((point_19[0] - point_15[0])**2 + (point_19[1] - point_15[1])**2)
            
            # Calculate the width between points 14 and 15
            width_18_14 = np.sqrt((point_18[0] - point_14[0])**2 + (point_18[1] - point_14[1])**2)
            
            print('>>>>>>>>>>>>>>>> length of 18 and 14: ', width_18_14)
            print('>>>>>>>>>>>>>>>> length of 19 and 15: ', width_19_15)
            print('\n19, 15, 18, 14 origranl', [point_19, point_15, point_18, point_14])
    
            point_19[1] = int(point_19[1] - 2 * width_19_15)
            point_18[1] = int(point_18[1] - 2 * width_18_14)
            
            print('19, 15, 18, 14 decreamented\n', [point_19, point_15, point_18, point_14])

            filter_kpts[-1] = point_19 
            filter_kpts[-2] = point_18
        else:
            pass
            
        # cap = cv2.imread(accessories_img_path, cv2.IMREAD_UNCHANGED)
        cap_h,cap_w =  cap.shape[:2]

        print('******** new filter_kpts: ', filter_kpts)
        
        
        head_coordinates = [item for item in filter_kpts if item[2] in points_to_check]
        head_coordinates_position = sorted(head_coordinates, key=lambda x: x[2], reverse=True)
        print('-------------------- head coordinates position with keypoint numbers: ', head_coordinates_position)
        print('-------------------- length of head coordinates position: ', len(head_coordinates_position))
        
        head_coordinates_position = [[item[0], item[1] + inc] for item in head_coordinates_position]
        print('-------------------- head coordinates position without keypoint numbers: ', head_coordinates_position)
        
        pts1=np.float32([[0,0],[cap_w,0],[0,cap_h],[cap_w,cap_h]])
        pts2=np.float32(head_coordinates_position)
        
        h, mask = cv2.findHomography(pts1, pts2, cv2.RANSAC,5.0)
    
        height, width, channels = animal_image.shape
        im1Reg = cv2.warpPerspective(cap, h, (width, height))
        animal_image = cv2.cvtColor(animal_image, cv2.COLOR_BGR2RGB) 
        animal_image_result = cvzone.overlayPNG(animal_image, im1Reg, (0, 0))
        
        # cv2.imshow("Original Image", animal_image_result)
        # cv2.waitKey(0)
        # cv2.destroyAllWindows()

        return animal_image_result
    
    elif point_14_exists and point_15_exists:

        
        # get value of 14 and 15
        point_14 = next(sublist for sublist in filter_kpts if sublist[2] == 14)
        point_15 = next(sublist for sublist in filter_kpts if sublist[2] == 15)
        
        print("********************* Both 14 and 15 exist.")
        

        
        # Calculate the width between points 14 and 15
        width_14_15 = np.sqrt((point_14[0] - point_15[0])**2 + (point_14[1] - point_15[1])**2)
        resized_accessories_img  = resize_with_aspect_ratio(cap, int(width_14_15))

        
        print(f"********************* The width between points 14 and 15 is: {width_14_15}")
        animal_image = cv2.cvtColor(animal_image, cv2.COLOR_RGB2BGR) 

        print('%%%%%%%%%%% shape of png', resized_accessories_img.shape)
        

        percentage_to_subtract = 38 
        offset_y = int(percentage_to_subtract / 100 * resized_accessories_img.shape[1])
        
        animal_image_result = cvzone.overlayPNG(animal_image, resized_accessories_img, (point_15[0], point_15[1] - offset_y ))

        
        # cv2.imshow("Original Image", animal_image_result)
        # cv2.waitKey(0)
        # cv2.destroyAllWindows()

        return animal_image_result
    ################# Check two points code end
    else:
        missing_points = [point for point in points_to_check if point not in [sublist[2] for sublist in filter_kpts]]
        print(f" ************The following points are missing: {missing_points}")
############################### cap working end ###############################

In [6]:
from ultralytics import YOLO
import cv2
from matplotlib import pyplot as plt
import os
import numpy as np
import pandas as pd
import cvzone
from PIL import Image 

model_path = 'animal-tryon/best.pt'
keywords_file = "animal-tryon/keypoint_definitions.csv"
animal_img = 'animal-tryon/input/dog1.jpg'
# animal_img = 'animal-tryon/images_data/frame_56.png'

accessories_img = 'animal-tryon/accessories/edited_images/07 Sky & Cloud/เหลือง3.png'
result_png = '4. Sky & Cloud(Product result).png'


ann_meta_data = pd.read_csv(keywords_file)
COLORS = ann_meta_data["Hex colour"].values.tolist()

COLORS_RGB_MAP = []
for color in COLORS:
    R, G, B = int(color[:2], 16), int(color[2:4], 16), int(color[4:], 16)
    COLORS_RGB_MAP.append({color: (R,G,B)})

model = YOLO(model_path)

BOX_IOU_THRESH = 0.55
BOX_CONF_THRESH=0.30
KPT_CONF_THRESH=0.68

inc = 15



animal_image = cv2.imread(animal_img)
animal_image = cv2.cvtColor(animal_image, cv2.COLOR_BGR2RGB)

cap = cv2.imread(accessories_img, cv2.IMREAD_UNCHANGED)

results = model.predict(animal_img, conf=BOX_CONF_THRESH, iou=BOX_IOU_THRESH)[0].cpu()

if not len(results.boxes.xyxy):
    animal_image

# Get the predicted boxes, conf scores and keypoints.
pred_boxes = results.boxes.xyxy.numpy()
pred_box_conf = results.boxes.conf.numpy()
pred_kpts_xy = results.keypoints.xy.numpy()
pred_kpts_conf = results.keypoints.conf.numpy()

# Draw predicted bounding boxes, conf scores and keypoints on image.
for boxes, score, kpts, confs in zip(pred_boxes, pred_box_conf, pred_kpts_xy, pred_kpts_conf):
    kpts_ids = np.where(confs > KPT_CONF_THRESH)[0]
    filter_kpts = kpts[kpts_ids]
    filter_kpts = np.concatenate([filter_kpts, np.expand_dims(kpts_ids, axis=-1)], axis=-1)
    
    # filter_kpts = filter_kpts.astype("int").tolist()
    filter_kpts = [[int(x) for x in inner_list] for inner_list in filter_kpts]
    print('******** old filter_kpts: ', filter_kpts)
    # animal_image = draw_landmarks(animal_image, filter_kpts)
    cap_wear_result = wear_cap(cap, animal_image, filter_kpts)
    # Check if the resulting image has valid dimensions
    if cap_wear_result is not None and cap_wear_result.shape[0] > 0 and cap_wear_result.shape[1] > 0:
        cv2.imwrite(result_png, cap_wear_result) 
        cv2.imshow("cap_wear_result", cap_wear_result)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    else:
        print("Error: Invalid dimensions for the resulting image.")

        # print("********************* Neither 14 nor 15 exists.")
    ################ 


image 1/1 D:\closet\tryon\animal-tryon\input\dog1.jpg: 640x448 1 dog, 361.8ms
Speed: 3.0ms preprocess, 361.8ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 448)
******** old filter_kpts:  [[432, 906, 0], [402, 794, 1], [391, 613, 2], [202, 895, 6], [224, 785, 7], [227, 606, 8], [423, 164, 14], [304, 147, 15], [343, 322, 16], [341, 378, 17], [450, 94, 18], [294, 73, 19]]
----------****** four points exits ******----------True
************ All points exist.
******** new filter_kpts:  [[432, 906, 0], [402, 794, 1], [391, 613, 2], [202, 895, 6], [224, 785, 7], [227, 606, 8], [423, 164, 14], [304, 147, 15], [343, 322, 16], [341, 378, 17], [450, 94, 18], [294, 73, 19]]
-------------------- head coordinates position with keypoint numbers:  [[294, 73, 19], [450, 94, 18], [304, 147, 15], [423, 164, 14]]
-------------------- length of head coordinates position:  4
-------------------- head coordinates position without keypoint numbers:  [[294, 88], [450, 109], [304, 162], [423, 1

# Working on videos

In [ ]:
from ultralytics import YOLO
import cv2
from matplotlib import pyplot as plt
import os
import numpy as np
import pandas as pd
import cvzone
from PIL import Image 

model_path = 'animal-tryon/best.pt'
keywords_file = "animal-tryon/keypoint_definitions.csv"

accessories_img = 'animal-tryon/accessories/hat1.png'
result_png = 'png_result.png'

ann_meta_data = pd.read_csv(keywords_file)
COLORS = ann_meta_data["Hex colour"].values.tolist()

COLORS_RGB_MAP = []
for color in COLORS:
    R, G, B = int(color[:2], 16), int(color[2:4], 16), int(color[4:], 16)
    COLORS_RGB_MAP.append({color: (R,G,B)})

model = YOLO(model_path)

BOX_IOU_THRESH = 0.55
BOX_CONF_THRESH=0.30
KPT_CONF_THRESH=0.68

inc = 15

def wear_cap(accessories_img_path,animal_image,  filter_kpts):
    ############################### cap working ############################### 

    ################# Check two points (if 14 and 15 exist in the third item of each list)
    
    point_14_exists = any(14 == sublist[2] for sublist in filter_kpts)
    point_15_exists = any(15 == sublist[2] for sublist in filter_kpts)


    

    ################ check four points 

    # Points to check
    points_to_check = [14, 15, 19, 18]
    
    # Check if all points exist in the third item of any list
    all_points_exist = all(point in [sublist[2] for sublist in filter_kpts] for point in points_to_check)
    print(f'----------****** four points exits ******----------{all_points_exist}', )
    if all_points_exist:
        print("************ All points exist.")
        

        ## *********** check if 19 > 15 and 18 > 14 (condition to correct image flip )
        # get value of 19 and 15
        point_19 = next(sublist for sublist in filter_kpts if sublist[2] == 19)
        point_15 = next(sublist for sublist in filter_kpts if sublist[2] == 15)

        if point_19[1] > point_15[1]:
            
            
    
    
            # get value of 18 and 14
            point_18 = next(sublist for sublist in filter_kpts if sublist[2] == 18)
            point_14 = next(sublist for sublist in filter_kpts if sublist[2] == 14)

            # Calculate the width between points 14 and 15
            width_19_15 = np.sqrt((point_19[0] - point_15[0])**2 + (point_19[1] - point_15[1])**2)
            
            # Calculate the width between points 14 and 15
            width_18_14 = np.sqrt((point_18[0] - point_14[0])**2 + (point_18[1] - point_14[1])**2)
            
            print('>>>>>>>>>>>>>>>> length of 18 and 14: ', width_18_14)
            print('>>>>>>>>>>>>>>>> length of 19 and 15: ', width_19_15)
            print('\n19, 15, 18, 14 origranl', [point_19, point_15, point_18, point_14])
    
            point_19[1] = int(point_19[1] - 2 * width_19_15)
            point_18[1] = int(point_18[1] - 2 * width_18_14)
            
            print('19, 15, 18, 14 decreamented\n', [point_19, point_15, point_18, point_14])

            filter_kpts[-1] = point_19 
            filter_kpts[-2] = point_18
        else:
            pass
            
        cap = cv2.imread(accessories_img_path, cv2.IMREAD_UNCHANGED)
        cap_h,cap_w =  cap.shape[:2]

        print('******** new filter_kpts: ', filter_kpts)
        
        
        head_coordinates = [item for item in filter_kpts if item[2] in points_to_check]
        head_coordinates_position = sorted(head_coordinates, key=lambda x: x[2], reverse=True)
        print('-------------------- head coordinates position with keypoint numbers: ', head_coordinates_position)
        print('-------------------- length of head coordinates position: ', len(head_coordinates_position))
        
        head_coordinates_position = [[item[0], item[1] + inc] for item in head_coordinates_position]
        print('-------------------- head coordinates position without keypoint numbers: ', head_coordinates_position)
        
        pts1=np.float32([[0,0],[cap_w,0],[0,cap_h],[cap_w,cap_h]])
        pts2=np.float32(head_coordinates_position)
        
        h, mask = cv2.findHomography(pts1, pts2, cv2.RANSAC,5.0)
    
        height, width, channels = animal_image.shape
        im1Reg = cv2.warpPerspective(cap, h, (width, height))
        animal_image = cv2.cvtColor(animal_image, cv2.COLOR_BGR2RGB) 
        animal_image_result = cvzone.overlayPNG(animal_image, im1Reg, (0, 0))
        
        # cv2.imshow("Original Image", animal_image_result)
        # cv2.waitKey(0)
        # cv2.destroyAllWindows()

        return animal_image_result
    
    elif point_14_exists and point_15_exists:

        
        # get value of 14 and 15
        point_14 = next(sublist for sublist in filter_kpts if sublist[2] == 14)
        point_15 = next(sublist for sublist in filter_kpts if sublist[2] == 15)
        
        print("********************* Both 14 and 15 exist.")
        

        
        # Calculate the width between points 14 and 15
        width_14_15 = np.sqrt((point_14[0] - point_15[0])**2 + (point_14[1] - point_15[1])**2)
        resized_accessories_img  = resize_with_aspect_ratio(accessories_img_path, int(width_14_15))

        
        print(f"********************* The width between points 14 and 15 is: {width_14_15}")
        animal_image = cv2.cvtColor(animal_image, cv2.COLOR_RGB2BGR) 

        print('%%%%%%%%%%% shape of png', resized_accessories_img.shape)
        

        percentage_to_subtract = 38 
        offset_y = int(percentage_to_subtract / 100 * resized_accessories_img.shape[1])
        
        animal_image_result = cvzone.overlayPNG(animal_image, resized_accessories_img, (point_15[0], point_15[1] - offset_y ))

        
        # cv2.imshow("Original Image", animal_image_result)
        # cv2.waitKey(0)
        # cv2.destroyAllWindows()

        return animal_image_result
    ################# Check two points code end
    else:
        missing_points = [point for point in points_to_check if point not in [sublist[2] for sublist in filter_kpts]]
        print(f" ************The following points are missing: {missing_points}")
    ############################### cap working end ###############################


# Open the video file
video_path = 'animal-tryon/input/dog_video2.mp4'
cap = cv2.VideoCapture(video_path)

# # Get the frames per second (fps) of the video
# fps = cap.get(cv2.CAP_PROP_FPS)

# # We need to set resolutions. 
# # so, convert them from float to integer. 
# frame_width = int(cap.get(3)) 
# frame_height = int(cap.get(4)) 
   
# size = (frame_width, frame_height) 
   
# Below VideoWriter object will create 
# a frame of above defined The output  
# is stored in 'filename.avi' file. 
# result = cv2.VideoWriter('filename.avi',  
#                          cv2.VideoWriter_fourcc(*'MJPG'), 
#                          10, size) 


# get FPS of input video 
fps = cap.get(cv2.CAP_PROP_FPS) 

# define output video and it's FPS 
output_file = 'output2.mp4'
output_fps = fps * 2
  
# define VideoWriter object 
fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
out = cv2.VideoWriter(output_file, fourcc, output_fps, 
                      (int(cap.get(3)), int(cap.get(4)))) 

counter  = 1
while cap.isOpened():
    ret, animal_image = cap.read()

    if not ret:
        break
    # animal_image = cv2.cvtColor(animal_image, cv2.COLOR_BGR2RGB)
    
    # results = model.predict(animal_image, conf=BOX_CONF_THRESH, ioQu=BOX_IOU_THRESH)[0].cpu()
    results = model.predict(animal_image, conf=BOX_CONF_THRESH, iou=BOX_IOU_THRESH)[0].cpu()

    if not len(results.boxes.xyxy):
        animal_image
    
    # Get the predicted boxes, conf scores and keypoints.
    pred_boxes = results.boxes.xyxy.numpy()
    pred_box_conf = results.boxes.conf.numpy()
    pred_kpts_xy = results.keypoints.xy.numpy()
    pred_kpts_conf = results.keypoints.conf.numpy()
    
    # Draw predicted bounding boxes, conf scores and keypoints on image.
    for boxes, score, kpts, confs in zip(pred_boxes, pred_box_conf, pred_kpts_xy, pred_kpts_conf):
        kpts_ids = np.where(confs > KPT_CONF_THRESH)[0]
        filter_kpts = kpts[kpts_ids]
        filter_kpts = np.concatenate([filter_kpts, np.expand_dims(kpts_ids, axis=-1)], axis=-1)
        
        # filter_kpts = filter_kpts.astype("int").tolist()
        filter_kpts = [[int(x) for x in inner_list] for inner_list in filter_kpts]
        print('******** filter_kpts: ', filter_kpts)

    animal_image = cv2.cvtColor(animal_image, cv2.COLOR_BGR2RGB)
    # animal_image = draw_landmarks(animal_image, filter_kpts)
    cap_wear_result = wear_cap(accessories_img, animal_image, filter_kpts)
    
    # Write the frame to the output video
    out.write(cap_wear_result) 
    
    # Show the frame
    # cv2.imshow("Original Image", cap_wear_result)

    # Break the loop if 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    # Wait to achieve the desired frame rate (30fps)
    cv2.waitKey(int(1000 / fps))
    print(f'*********************** {counter} frames done ***********************')
    counter+=1


# Release the video capture object and close all windows
out.release()
cap.release()
cv2.destroyAllWindows()